# How to Connect to Coinbase REST API and Display Bitcoin Historic Data?

by Nils Ackermann

## Introduction

In case you want to get started as a crypto investor and you want to take matters into your own hands, it is important that you are able to connect to a currency exchange such as [pro.coinbase.com](https://pro.coinbase.com). This is the essential first step to start data analysis and automation and move into algorithmic trading. In this notebook you will learn the following:

- How to connect to the public coinbase.com pro REST API via Python
- How to load data from coinbase into Pandas
- How to visualize Bitcoin (or any other crypto currency) historic data in Python

The example uses Coinbase Pro as an exchange but it should be easily transferable to the other major digital currency exchange [Kraken](https://www.kraken.com). The full pro.coinbase.com documentation can be found [here](https://docs.pro.coinbase.com).

## Step 0 - Imports, Libraries and Functions

For the example, we need the following python imports:

- **Requests:** For connecting to the REST endpoints from Coinbase Pro, we will use [this](http://docs.python-requests.org/en/master/) great HTTP library. Very user friendly. Only a few lines of code to connect via REST. 
- **Pandas:** For data analysis and manipulation we use [Pandas](https://pandas.pydata.org).
- **Json:** We are dealing with json format, therefore let's import it. The documentation is [here](https://docs.python.org/3/library/json.html)
- **Datetime:** We are doing some smaller data calculations, therefore we need [Datetime](https://docs.python.org/3/library/datetime.html).
- **Matplotlib:** At the end of the notebook, we will visualize some data, therefore we need [this](https://matplotlib.org) library.
- **MPL Finance:** Formerly part of matplot lib, the finance module has been deprecated and is now in its own library. You can find more information [here](https://github.com/matplotlib/mplfinance). We use this library to do the financial plotting at the end of the notebook. There is also an older version called mpl-finance. In case you have worked with mpl-finance before, I have also included the code in the backup at the end of the notebook.
- **Seaborn:** For a nicer visualization of the plots, we will use [Seaborn](https://seaborn.pydata.org).

Since we will reuse some of the code throughout the notebook, we will define one functions first:

- connect(url,param): this function takes the URL and optionally the parameters to a REST endpoint as an input
. It then returns the response. If an error occurs, an exception is thrown and the error is printed.

In [ ]:
import requests
from requests.exceptions import HTTPError
import pandas as pd
import json as js
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
# Uncomment this if you like to use the old MPL library
#from mpl_finance import candlestick_ohlc
import mplfinance as mpf
import matplotlib.dates as mpl_dates
import matplotlib.ticker as tkr
%matplotlib inline
import seaborn as sns

In [ ]:
# Set some standard parameters upfront
pd.options.display.float_format = '{:.1f}'.format
sns.set() # default seaborn look and feel
plt.style.use('ggplot')
REST_API = 'https://api.pro.coinbase.com'
PRODUCTS = REST_API+'/products'
# I am only interested in a few currencies that I want to trade, so let's add them here:
MY_CURRENCIES = ['BTC-EUR','ETH-EUR','LTC-EUR','BCH-EUR'] 


In [ ]:
def connect(url, *args, **kwargs):
    try:
        if kwargs.get('param', None) is not None:
            response = requests.get(url,params)
        else:
            response = requests.get(url)
        response.raise_for_status()
        #print('HTTP connection success!')
        return response
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

## Step 1 - Connect to REST Endpoint

Connect to the Coinbase REST endpoint (/products) to retrieve all products that are currently available for trading on Coinbase Pro. We are using a try / except block in the function defined above to react to potential connectivity issues.

In [ ]:
response = connect(PRODUCTS)
response_content = response.content
response_text = response.text
response_headers = response.headers

## Step 2 - Get Basic Currency Information

Let's print some basic information about the currencies that I have downselected above: Bitcoin, Ethereum, Litecoin and Bitcoin Cash.

In [ ]:
df_currencies = pd.read_json (response_text)
print("\nNumber of columns in the dataframe: %i" % (df_currencies.shape[1]))
print("Number of rows in the dataframe: %i\n" % (df_currencies.shape[0]))
columns = list(df_currencies.columns)
print(columns)
print() 
df_currencies[df_currencies.id.isin(MY_CURRENCIES)][['id', 'quote_currency', 'base_min_size', 'base_max_size']].head(5)

## Step 3 - Get More Statistics

Ok - great. We see that we are able to connect to the Coinbase Pro API with only a few lines of code. Next, we are interested in what has happened over the last 24 hours for each of our favorite currency:
- Open: the opening value in EUR
- High: the highest price in EUR
- Low: the lowest price in EUR
- Last: the current price in EUR
- Volume: volume of trades

In [ ]:
currency_rows = []
for currency in MY_CURRENCIES:
    response = connect(PRODUCTS+'/'+currency+'/stats')
    response_content = response.content
    data = js.loads(response_content.decode('utf-8'))
    currency_rows.append(data)
# Create dataframe and set row index as currency name
df_statistics = pd.DataFrame(currency_rows, index = MY_CURRENCIES)
df_statistics

## Step 4: Drill Down Into Bitcoin Historic Data

We now want to find out how the price has developed over the last 90 days. We will use Bitcoin as an example. For now, a simple table view of the data is good enough for us. 

In [ ]:
start_date = (datetime.today() - timedelta(days=90)).isoformat()
end_date = datetime.now().isoformat()
# Please refer to the coinbase documentation on the expected parameters
params = {'start':start_date, 'end':end_date, 'granularity':'86400'}
response = connect(PRODUCTS+'/BTC-EUR/candles', param = params)
response_text = response.text
df_history = pd.read_json(response_text)
# Add column names in line with the Coinbase Pro documentation
df_history.columns = ['time','low','high','open','close','volume']


In [ ]:
# We will add a few more columns just for better readability
df_history['date'] = pd.to_datetime(df_history['time'], unit='s')
df_history['year'] = pd.DatetimeIndex(df_history['date']).year
df_history['month'] = pd.DatetimeIndex(df_history['date']).month
df_history['day'] = pd.DatetimeIndex(df_history['date']).day
# Only display the first 5 rows
df_history.head(5).drop(['time','date'], axis=1)

## Step 5 - Candlestick Chart

As the final step, let's view the data as a candlestick chart. As a bonus, the chart is also enhanced with the moving average and the total volume traded within each time slice. And all of this with only a few lines of code. How great is that!

In [ ]:
# Make a copy of the original dataframe
df_ohlc = df_history
# Remove unnecessary columns and only show the last 30 days
df_ohlc = df_ohlc.drop(['time','year','month','day'], axis = 1).head(30)
# Columns must be in a specific order for the candlestick chart (OHLC)
df_ohlc = df_ohlc[['date', 'open', 'high', 'low', 'close','volume']]
# Index must be set as the date
df_ohlc.set_index('date', inplace=True)
# Inverse order is expected so let's reverse the rows in the dataframe
df_ohlc = df_ohlc[::-1]
mpf.plot(df_ohlc,type='candle',mav=(3,6,9),volume=True)

## Wrap Up

You made it to the end of the notebook. Thanks a lot for taking the time to go through the code. Maybe the fire is ignited and you will start coding your own solution on top of the great digital currency APIs out there. 

Go build!

## Backup - Previous MLP_Finance Library

In case you are still familiar with the older mlp_finance library, you could also use the following code. Don't forget to uncomment the import statement at the beginning of the notebook.

In [ ]:
# Make a copy of the original dataframe
df_ohlc = df_history
# Remove unnecessary columns and only show the last 30 days
df_ohlc = df_ohlc.drop(['volume','time','year','month','day'], axis = 1).head(30)
# Columns must be in a specific order for the candlestick chart (OHLC)
df_ohlc = df_ohlc[['date', 'open', 'high', 'low', 'close']]
# Date must also be in a specific format for MPL
df_ohlc['date'] = df_ohlc['date'].apply(mpl_dates.date2num)
df_ohlc = df_ohlc.astype(float)
# Add simple moving average for 5 days
df_ohlc['SMA5'] = df_ohlc['close'].rolling(5).mean()

In [ ]:
# Create the candlestick chart
fig, ax = plt.subplots()
fig.set_figheight(6)
fig.set_figwidth(8)
candlestick_ohlc(ax, df_ohlc.values, width=0.5, colorup='green', colordown='red')
ax.plot(df_ohlc['date'], df_ohlc['SMA5'], color='blue', label='SMA5')

# Set chart title and labels
ax.set_xlabel('Date')
ax.set_ylabel('Price (EUR)')
fig.suptitle('Bitcoin - Daily Candlestick Chart')

# Format date on x axis and number on y axis
date_format = mpl_dates.DateFormatter('%m-%d-%Y')
ax.xaxis.set_major_formatter(date_format)
ax.yaxis.set_major_formatter(tkr.FuncFormatter(lambda x, p: format(int(x), ',')))

plt.legend()

plt.show()